<h5 style="color:rgba(0, 0, 178, 0.6);">This code hidden warrnings</h5>

In [1]:
import warnings
warnings.filterwarnings('ignore')

<h5 style="color:rgba(0, 0, 178, 0.6);">Install librarys that we need</h5>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("bmh")
from tensorflow import keras
from sklearn.cluster import KMeans
import random
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
tf.__version__

'2.15.0'

<h5 style="color:rgba(0, 0, 178, 0.6);">Read the data from data set</h5>

In [3]:
df = pd.read_csv("../../DATA/Data_increase.csv").drop(["Unnamed: 0","Unnamed: 0.1"], axis=1)
df["LLD-LLS"] = df.log_LLD - df.log_LLS
df["GR-GKUT"] = df.GR - df.GKUT
df

DEPT       GR       GKUT      NPHI      PE    RHOB      DRHO  \
0      2705.2524  19.0270  18.728100  0.051300  5.2064  2.6527  0.019800   
1      2705.4048  15.8497  16.507000  0.046800  5.2632  2.6564  0.022900   
2      2705.5572  13.3455  14.392100  0.041100  5.2742  2.6546  0.021800   
3      2705.7096  12.2315  13.696900  0.036900  5.2543  2.6488  0.020200   
4      2705.8620  13.0494  13.258900  0.032200  5.1625  2.6475  0.014500   
...          ...      ...        ...       ...     ...     ...       ...   
17456  2979.2676  24.4099  33.716596  0.368062  4.5393  2.2819  0.037705   
17457  2979.4200  24.4099  33.998501  0.368062  4.4872  2.2728  0.038547   
17458  2979.5724  24.4099  34.077848  0.368062  4.4662  2.2730  0.038599   
17459  2979.7248  24.4099  33.842925  0.368062  4.3735  2.2731  0.037318   
17460  2979.8772  24.4099  33.796072  0.368062  4.3489  2.2674  0.035771   

       well   log_LLS   log_LLD   LLD-LLS   GR-GKUT  
0         1  0.699109  0.749381  0.050272  0.298900  
1         1  0.708140  0.769960  0.061820 -0.657300  
2         1  0.751779  0.823416  0.071636 -1.046600  
3         1  0.787269  0.865447  0.078178 -1.465400  
4         1  0.833886  0.921738  0.087852 -0.209500  
...     ...       ...       ...       ...       ...  
17456     3 -0.272377 -0.134245  0.138133 -9.306696  
17457     3 -0.269783 -0.132356  0.137427 -9.588601  
17458     3 -0.263922 -0.128077  0.135845 -9.667948  
17459     3 -0.261378 -0.126215  0.135163 -9.433025  
17460     3 -0.259558 -0.124881  0.134678 -9.386172  

[17461 rows x 12 columns]

<h5 style="color:rgba(0, 0, 178, 0.6);">Split data to; train and test ; X and y </h5>

In [4]:
train = df[~df['well'].isin(df.groupby("well").get_group(6)['well'])]
train = train[~train['well'].isin(train.groupby("well").get_group(4)['well'])]
test = df.groupby("well").get_group(6)
val = df.groupby("well").get_group(4)

X_train = train.drop(["NPHI", "RHOB","DRHO", "well"], axis=1)
y_train = train["NPHI"]

X_test = test.drop(["NPHI", "RHOB", "DRHO", "well"], axis=1)
y_test = test["NPHI"]

X_val = val.drop(["NPHI", "RHOB", "DRHO", "well"], axis=1)
y_val = val["NPHI"]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

<h5 style="color:rgba(0, 0, 178, 0.6);">Create a function that build a dynamic model that can change the shape of the NN model with hyper parameters(hp) parameters</h5>

In [16]:
# hp ======> hyper parameters = [number_nuron_layer1, number_nuron_layer2, number_nuron_layer3,
#                          number_nuron_layer4, Activation_fuctin_layer1, Activaiton_function_layer2,
#                     Activation_fuctin_layer3, Activation_fuctin_layer4, If for layer1 have Drop_out=1 else 0,
#         If for layer2 have Drop_out=1 else 0, If for layer3 have Drop_out=1 else 0, If for layer4 have Drop_out=1 else 0,
#     If for layer1 have batch_norm=1 else 0, If for layer2 have batch_norm=1 else 0, If for layer1 have batch_norm=3 else 0,
#                       If for layer1 have batch_norm=1 else 0, learning_rate, batch_size]


# in this function we create a function that can build model that can chane evry time

def dynamic_model(hp, train=train):

    
    activation_function = ["relu","sigmoid","softmax","softsign",
                           "tanh","selu","elu" ,"exponential",
                           "leaky_relu","relu6","silu","gelu",
                           "hard_sigmoid","linear","mish","log_softmax"]
    
    
    input_layer = keras.layers.Input(shape=(8))
    layer1 = keras.layers.Dense(hp[0], activation_function[hp[4]], name="layer1")(input_layer)
    
#     in this sectino we have 4 layer  so that we have 4 section for dscetion that we have in each layer DropOut or BatchNorm
#   this is layer1 we have 4 condition:
    if ((hp[8] == 1) & (hp[12] == 1)):
        drop1 = keras.layers.Dropout(0.3)(layer1)
        norm1 = keras.layers.BatchNormalization()(drop1)
        layer2 = keras.layers.Dense(hp[1], activation_function[hp[5]], name="layer2")(norm1)
    elif ((hp[8] == 1) & (hp[12] == 0)):
        drop1 = keras.layers.Dropout(0.3)(layer1)
        layer2 = keras.layers.Dense(hp[1], activation_function[hp[5]], name="layer2")(drop1)
    elif ((hp[8] == 0) & (hp[12] == 1)):
        norm1 = keras.layers.BatchNormalization()(layer1)
        layer2 = keras.layers.Dense(hp[1], activation_function[hp[5]], name="layer2")(norm1)
    else:
        layer2 = keras.layers.Dense(hp[1], activation_function[hp[5]], name="layer2")(layer1)
        
#    layer2: 
    if ((hp[9] == 1) & (hp[13] == 1)):
        drop2 = keras.layers.Dropout(0.3)(layer2)
        norm2 = keras.layers.BatchNormalization()(drop2)
        layer3 = keras.layers.Dense(hp[2], activation_function[hp[6]], name="layer3")(norm2)
    elif ((hp[9] == 1) & (hp[13] == 0)):
        drop2 = keras.layers.Dropout(0.3)(layer2)
        layer3 = keras.layers.Dense(hp[2], activation_function[hp[6]], name="layer3")(drop2)
    elif ((hp[9] == 0) & (hp[13] == 1)):
        norm2 = keras.layers.BatchNormalization()(layer2)
        layer3 = keras.layers.Dense(hp[2], activation_function[hp[6]], name="layer3")(norm2)
    else:
        layer3 = keras.layers.Dense(hp[2], activation_function[hp[6]], name="layer3")(layer2)
        
#     layer3 :   
    if ((hp[10] == 1) & (hp[14] == 1)):
        drop3 = keras.layers.Dropout(0.3)(layer3)
        norm3 = keras.layers.BatchNormalization()(drop3)
        layer4 = keras.layers.Dense(hp[3], activation_function[hp[7]], name="layer4")(norm3)
    elif ((hp[10] == 1) & (hp[14] == 0)):
        drop3 = keras.layers.Dropout(0.3)(layer3)
        layer4 = keras.layers.Dense(hp[3], activation_function[hp[7]], name="layer4")(drop3)
    elif ((hp[10] == 0) & (hp[14] == 1)):
        norm3 = keras.layers.BatchNormalization()(layer3)
        layer4 = keras.layers.Dense(hp[3], activation_function[hp[7]], name="layer4")(norm3)
    else:
        layer4 = keras.layers.Dense(hp[3], activation_function[hp[7]], name="layer4")(layer3)
        
#    layer4:
    if ((hp[11] == 1) & (hp[15] == 1)):
        drop4 = keras.layers.Dropout(0.3)(layer4)
        norm4 = keras.layers.BatchNormalization()(drop4)
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(norm4)
    elif ((hp[11] == 1) & (hp[15] == 0)):
        drop4 = keras.layers.Dropout(0.3)(layer4)
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(drop4)
    elif ((hp[11] == 0) & (hp[15] == 1)):
        norm4 = keras.layers.BatchNormalization()(layer4)
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(norm4)
    else:
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(layer4)
        
#     at last we create final model
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    
#     in this section we complie the model
    optimizer =keras.optimizers.Adamax(learning_rate=hp[16])
    model.compile(loss="mae", optimizer=optimizer, metrics=["mse"])

    # wright the callbacks that we need
#     checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_checkpoints.h5", save_weights_only=True,best_only=True)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)
    
    # fit the model
    history = model.fit(X_train,
                         y_train, 
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping_cb],
                        batch_size=hp[17],
                        epochs=1000,
                        verbose=0)
    
# for chace the model we prdiction x_val
    val_pred = model.predict(X_val)
#   because we r2_score dont work with nan values we chane nan value with -999
    val_pred[np.isnan(val_pred)] = -999
        
    solution_fitness = r2_score(y_val, val_pred)
            
    return model, solution_fitness


<h5 style="color:rgba(0, 0, 178, 0.6);">Create a function that build randomly DNA for obtimizer</h5>

In [17]:
def hp_creator(num_solutions=50):
    hps = pd.DataFrame(columns=[
                                "nurn1","nurn2","nurn3","nurn4",
                                "Active1","Active2","Active3","Active4",
                                "Drop1", "Drop2", "Drop3", "Drop4",
                                "Norm1", "Norm2", "Norm3", "Norm4",
                                "Learn_rate", "batch_size", "solution_fitness"
                               ])
    
    hps["nurn1"], hps["nurn2"] = [random.randint(10, 150) for _ in range(num_solutions)], [random.randint(10, 150) for _ in range(num_solutions)]
    hps["nurn3"], hps["nurn4"] = [random.randint(10, 150) for _ in range(num_solutions)], [random.randint(10, 150) for _ in range(num_solutions)]
    
    hps["Active1"], hps["Active2"] = [random.randint(0, 15) for _ in range(num_solutions)], [random.randint(0, 15) for _ in range(num_solutions)]
    hps["Active3"], hps["Active4"] = [random.randint(0, 15) for _ in range(num_solutions)], [random.randint(0, 15) for _ in range(num_solutions)]
                                                                                 
    hps["Drop1"], hps["Drop2"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    hps["Drop3"], hps["Drop4"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    
    hps["Norm1"], hps["Norm2"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    hps["Norm3"], hps["Norm4"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    
    hps["Learn_rate"] = [random.choice([0.001, 0.01, 0.0001, 0.00001, 0.1, 0.000001, 0.0000001]) for _ in range(num_solutions)]
    hps["batch_size"] = [random.randint(30, 300) for _ in range(num_solutions)]
    

    return hps


<h5 style="color:rgba(0, 0, 178, 0.6);">Create a function that build child DNA from best DNA's for obtimizer</h5>

In [18]:
def child_creator(parents, num_solutions=50):
    hps = pd.DataFrame(columns=[
                                "nurn1","nurn2","nurn3","nurn4",
                                "Active1","Active2","Active3","Active4",
                                "Drop1", "Drop2", "Drop3", "Drop4",
                                "Norm1", "Norm2", "Norm3", "Norm4",
                                "Learn_rate", "batch_size", "solution_fitness"
                               ])
    
    hps["nurn1"], hps["nurn2"] = [random.choice(parents["nurn1"].values) for _ in range(num_solutions)], [random.choice(parents["nurn2"].values) for _ in range(num_solutions)]
    hps["nurn3"], hps["nurn4"] = [random.choice(parents["nurn3"].values) for _ in range(num_solutions)], [random.choice(parents["nurn4"].values) for _ in range(num_solutions)]
    
    hps["Active1"], hps["Active2"] = [random.choice(parents["Active1"].values) for _ in range(num_solutions)], [random.choice(parents["Active2"].values) for _ in range(num_solutions)]
    hps["Active3"], hps["Active4"] = [random.choice(parents["Active3"].values) for _ in range(num_solutions)], [random.choice(parents["Active4"].values) for _ in range(num_solutions)]
                                                                                 
    hps["Drop1"], hps["Drop2"] = [random.choice(parents["Drop1"].values) for _ in range(num_solutions)], [random.choice(parents["Drop2"].values) for _ in range(num_solutions)]
    hps["Drop3"], hps["Drop4"] = [random.choice(parents["Drop3"].values) for _ in range(num_solutions)], [random.choice(parents["Drop4"].values) for _ in range(num_solutions)]
    
    hps["Norm1"], hps["Norm2"] = [random.choice(parents["Norm1"].values) for _ in range(num_solutions)], [random.choice(parents["Norm2"].values) for _ in range(num_solutions)]
    hps["Norm3"], hps["Norm4"] = [random.choice(parents["Norm3"].values) for _ in range(num_solutions)], [random.choice(parents["Norm4"].values) for _ in range(num_solutions)]
    
    hps["Learn_rate"] = [random.choice(parents["Learn_rate"].values) for _ in range(num_solutions)]
    hps["batch_size"] = [random.choice(parents["batch_size"].values) for _ in range(num_solutions)]
    
    hps = pd.concat([hps, parents], axis=0).reset_index().drop("index", axis=1)
    
    
    return hps

<h5 style="color:rgba(0, 0, 178, 0.6);">we use obtimizer and dynamic model to find best DNA</h5>

In [19]:
# we create a DataFrame to save best solutions in it and create child from this best slutions
best_slutions = pd.DataFrame(columns=[
                                "nurn1","nurn2","nurn3","nurn4",
                                "Active1","Active2","Active3","Active4",
                                "Drop1", "Drop2", "Drop3", "Drop4",
                                "Norm1", "Norm2", "Norm3", "Norm4",
                                "Learn_rate", "batch_size", "solution_fitness"
                               ])
# we create 5 times cheild
for i in range(5):
    #for start our work we shoud create random DNA
    if i == 0:
        #_____________________________________________________________________________________
        #create 100 DNA randomly for nex step
        hps = hp_creator(num_solutions=100)
        #check each Random DNAs and bild models to see which models is better
        for j in range(len(hps)):
            _, hps["solution_fitness"][j] = dynamic_model(hps.iloc[j,:])
            print("step: ", j+1, f"    solution_fitness_random{j}: ", hps["solution_fitness"][j])
        #check each models and select that models have positive r2_score
        hps = hps[hps["solution_fitness"]>=0]
        #and sort the DNAs and select 25 best of this DNAs
        hps = hps.sort_values("solution_fitness").reset_index(drop=True)
        #save 25 of best to parents data frame and save the best solution to best_slution data frame
        parents = hps.iloc[len(hps)-25:,:]
        best_slutions = pd.concat([best_slutions, parents.iloc[len(parents)-1:len(parents),:]], axis=0)
        #_______________________________________________________________________________________________
    
    #_____________________________________________________________________________________
    #now we create 100 childs solution form this parents slutions
    childs = child_creator(parents, num_solutions=100)
    for j in range(len(childs)):
        _, childs["solution_fitness"][j] = dynamic_model(childs.iloc[j,:])
        print("step: ", j+1, f"    solution_fitness_child{j}: ", childs["solution_fitness"][j])
        
    childs = childs[childs["solution_fitness"]>=0]
    childs = childs.sort_values("solution_fitness").reset_index(drop=True)
    
    parents = childs.iloc[len(childs)-25:,:]
    best_slutions = pd.concat([best_slutions, parents.iloc[len(parents)-1:,:]], axis=0)
    #___________________________________________________________________________________

# save the best solutions to hard
best_slutions.to_csv("bestmodels.csv")
        

38/38 [==============================] - 0s 6ms/step
step:  1     solution_fitness_random0:  0.30372427569942495
38/38 [==============================] - 0s 5ms/step
step:  2     solution_fitness_random1:  -5.53867361875891
38/38 [==============================] - 0s 5ms/step
step:  3     solution_fitness_random2:  -486.8620701467944
38/38 [==============================] - 0s 4ms/step
step:  4     solution_fitness_random3:  0.6389364164512146
38/38 [==============================] - 0s 4ms/step
step:  5     solution_fitness_random4:  0.6521034959785816
38/38 [==============================] - 0s 3ms/step
step:  6     solution_fitness_random5:  0.62180638059896
38/38 [==============================] - 0s 3ms/step
step:  7     solution_fitness_random6:  0.5614793762618038
38/38 [==============================] - 1s 8ms/step
step:  8     solution_fitness_random7:  0.6480373655004308
38/38 [==============================] - 1s 13ms/step
step:  9     solution_fitness_random8:  0.5378182688

38/38 [==============================] - 0s 2ms/step
step:  73     solution_fitness_random72:  0.5687709171456721
38/38 [==============================] - 0s 2ms/step
step:  74     solution_fitness_random73:  0.691556104666975
38/38 [==============================] - 0s 2ms/step
step:  75     solution_fitness_random74:  -10.248055769917244
38/38 [==============================] - 0s 2ms/step
step:  76     solution_fitness_random75:  0.48612810958256947
38/38 [==============================] - 0s 2ms/step
step:  77     solution_fitness_random76:  -0.8638566092290101
38/38 [==============================] - 0s 2ms/step
step:  78     solution_fitness_random77:  -291936953.7702268
38/38 [==============================] - 0s 2ms/step
step:  79     solution_fitness_random78:  0.7274165385990554
38/38 [==============================] - 0s 2ms/step
step:  80     solution_fitness_random79:  0.112040158602988
38/38 [==============================] - 0s 2ms/step
step:  81     solution_fitness_ran

38/38 [==============================] - 0s 2ms/step
step:  117     solution_fitness_child116:  0.6814409313607472
38/38 [==============================] - 0s 2ms/step
step:  118     solution_fitness_child117:  0.6560636408863514
38/38 [==============================] - 0s 2ms/step
step:  119     solution_fitness_child118:  0.6936911343676532
38/38 [==============================] - 0s 2ms/step
step:  120     solution_fitness_child119:  0.7101289697193369
38/38 [==============================] - 0s 2ms/step
step:  121     solution_fitness_child120:  0.7108409558784534
38/38 [==============================] - 0s 2ms/step
step:  122     solution_fitness_child121:  0.69494306582484
38/38 [==============================] - 0s 1ms/step
step:  123     solution_fitness_child122:  0.6951778315879198
38/38 [==============================] - 0s 1ms/step
step:  124     solution_fitness_child123:  0.6145817269549656
38/38 [==============================] - 0s 2ms/step
step:  125     solution_fitne

38/38 [==============================] - 0s 2ms/step
step:  65     solution_fitness_child64:  0.7045991495164545
38/38 [==============================] - 0s 2ms/step
step:  66     solution_fitness_child65:  0.6076529127837595
38/38 [==============================] - 0s 2ms/step
step:  67     solution_fitness_child66:  0.666481431905554
38/38 [==============================] - 0s 2ms/step
step:  68     solution_fitness_child67:  0.6577840725054603
38/38 [==============================] - 0s 2ms/step
step:  69     solution_fitness_child68:  0.5397524616212338
38/38 [==============================] - 0s 2ms/step
step:  70     solution_fitness_child69:  0.6743584958539732
38/38 [==============================] - 0s 2ms/step
step:  71     solution_fitness_child70:  0.6365243930535963
38/38 [==============================] - 0s 2ms/step
step:  72     solution_fitness_child71:  0.681333561288076
38/38 [==============================] - 0s 3ms/step
step:  73     solution_fitness_child72:  0.67

step:  84     solution_fitness_child83:  0.6842397170739563
38/38 [==============================] - 1s 5ms/step
step:  85     solution_fitness_child84:  0.6014276559114389
38/38 [==============================] - 1s 10ms/step
step:  86     solution_fitness_child85:  0.6934820047360686
38/38 [==============================] - 1s 8ms/step
step:  87     solution_fitness_child86:  0.03670335399142932
38/38 [==============================] - 0s 5ms/step
step:  88     solution_fitness_child87:  0.615548629171603
38/38 [==============================] - 0s 3ms/step
step:  89     solution_fitness_child88:  0.6977407061015539
38/38 [==============================] - 0s 3ms/step
step:  90     solution_fitness_child89:  0.5836282218161333
38/38 [==============================] - 0s 5ms/step
step:  91     solution_fitness_child90:  0.6848925682074032
38/38 [==============================] - 0s 2ms/step
step:  92     solution_fitness_child91:  0.7002320248760467
38/38 [============================

38/38 [==============================] - 0s 1ms/step
step:  32     solution_fitness_child31:  0.6846141917993387
38/38 [==============================] - 0s 1ms/step
step:  33     solution_fitness_child32:  0.6823524149328605
38/38 [==============================] - 0s 1ms/step
step:  34     solution_fitness_child33:  0.50724569392476
38/38 [==============================] - 0s 1ms/step
step:  35     solution_fitness_child34:  0.6961731623982121
38/38 [==============================] - 0s 1ms/step
step:  36     solution_fitness_child35:  0.7082823399748448
38/38 [==============================] - 0s 1ms/step
step:  37     solution_fitness_child36:  0.6944343489389952
38/38 [==============================] - 0s 1ms/step
step:  38     solution_fitness_child37:  0.6202776717655106
38/38 [==============================] - 0s 2ms/step
step:  39     solution_fitness_child38:  0.5832417440575084
38/38 [==============================] - 0s 1ms/step
step:  40     solution_fitness_child39:  0.65

38/38 [==============================] - 0s 2ms/step
step:  105     solution_fitness_child104:  0.7013570408904762
38/38 [==============================] - 0s 2ms/step
step:  106     solution_fitness_child105:  0.665074855552158
38/38 [==============================] - 0s 2ms/step
step:  107     solution_fitness_child106:  0.7090526663287219
38/38 [==============================] - 0s 2ms/step
step:  108     solution_fitness_child107:  0.6901149015649689
38/38 [==============================] - 0s 2ms/step
step:  109     solution_fitness_child108:  0.6963088739810452
38/38 [==============================] - 0s 2ms/step
step:  110     solution_fitness_child109:  0.7001755283772706
38/38 [==============================] - 0s 2ms/step
step:  111     solution_fitness_child110:  0.6936689296284492
38/38 [==============================] - 0s 2ms/step
step:  112     solution_fitness_child111:  0.7000712316238265
38/38 [==============================] - 0s 2ms/step
step:  113     solution_fitn

38/38 [==============================] - 0s 2ms/step
step:  124     solution_fitness_child123:  0.7045928145337232
38/38 [==============================] - 0s 1ms/step
step:  125     solution_fitness_child124:  0.6872490043571499


In [21]:
best_slutions

nurn1 nurn2 nurn3 nurn4 Active1 Active2 Active3 Active4 Drop1 Drop2 Drop3  \
61     97   137    76    12      12       5       9       9     0     1     0   
112    97   137    76    12      12       5       9       9     0     1     0   
123   135    25   149    82       3      14       2       3     1     1     0   
124    68    60   149    78      10       9      14       2     1     0     0   
124    64    19   149    91       1       7      13       4     0     1     1   
124    53    94   137    82      12       7       1       0     1     0     0   

    Drop4 Norm1 Norm2 Norm3 Norm4  Learn_rate batch_size solution_fitness  
61      0     0     1     0     1       0.010        220         0.727417  
112     0     0     1     0     1       0.010        220         0.724919  
123     1     0     1     1     0       0.010        135         0.731489  
124     0     0     1     1     1       0.010        205         0.710515  
124     0     0     1     0     0       0.010        205         0.717048  
124     0     1     1     1     1       0.001         50          0.72619